In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
from sklearn import preprocessing 

In [2]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [12]:
# root_path= '/content/gdrive/My Drive/MergeFileCrop.csv'
root_path='../data/Crop_recommendation.csv'
otherfile='../data/indiaselected.csv'

In [13]:
df=pd.read_csv(root_path)
of=pd.read_csv(otherfile)

In [14]:
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [15]:
crops=of['Crop'].str.lower().unique()
crops

array(['rice', 'banana', 'maize', 'grapes', 'mango', 'orange', 'papaya',
       'jute', 'blackgram', 'coffee', 'lentil', 'apple'], dtype=object)

In [16]:
df1=df[df['label'].isin(crops)].reset_index(drop=True)

In [17]:
df2=df1.drop(columns=['ph'])
le = preprocessing.LabelEncoder()
# df['Crop']=le.fit_transform(df['Crop Type'])
df2['Crop']=le.fit_transform(df2['label'])

In [18]:
df3=df2[df2['N']>0 & df2['P'] & df2['K']].reset_index(drop=True)
crop_dic=dict(zip(df3['Crop'],df3['label']))
pickle.dump(crop_dic, open("dictcrop.pkl", "wb"))
features = df3[['temperature', 'humidity','rainfall','Crop']]
target1 = df3['N']
target2=df3['P']
target3=df3['K']
df3.isna().any()


N              False
P              False
K              False
temperature    False
humidity       False
rainfall       False
label          False
Crop           False
dtype: bool

In [19]:
df3

,N,P,K,temperature,humidity,rainfall,label,Crop
0,90,42,43,20.879744,82.002744,202.935536,rice,11
1,85,58,41,21.770462,80.319644,226.655537,rice,11
2,60,55,44,23.004459,82.320763,263.964248,rice,11
3,74,35,40,26.491096,80.158363,242.864034,rice,11
4,78,42,42,20.130175,81.604873,262.717340,rice,11
...,...,...,...,...,...,...,...,...
1180,107,34,32,26.774637,66.413269,177.774507,coffee,3
1181,99,15,27,27.417112,56.636362,127.924610,coffee,3
1182,118,33,30,24.131797,67.225123,173.322839,coffee,3
1183,117,32,34,26.272418,52.127394,127.175293,coffee,3


In [11]:
import pickle
from sklearn.preprocessing import StandardScaler
scalerfile = 'scalernpk.sav'
scaler = StandardScaler()
scaler.fit(features)
pickle.dump(scaler, open(scalerfile, 'wb'))
scaled=scaler.transform(features)

NameError: name 'features' is not defined

In [26]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(scaled,np.log(target1),test_size = 0.2,random_state =2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(scaled,np.log(target2),test_size = 0.2,random_state =2)
X_train3, X_test3, y_train3, y_test3 = train_test_split(scaled,np.log(target3),test_size = 0.2,random_state =2)

In [39]:
from sklearn.ensemble import GradientBoostingRegressor

regressor1 = GradientBoostingRegressor(random_state=0)
regressor1.fit(X_train1,y_train1)
regressor2 = GradientBoostingRegressor(random_state=0)
regressor2.fit(X_train2,y_train2)
regressor3 = GradientBoostingRegressor(random_state=0)
regressor3.fit(X_train3,y_train3)
predicted_valuesn = regressor1.predict(X_test1)
predicted_valuesp = regressor2.predict(X_test2)
predicted_valuesk = regressor3.predict(X_test3)

print(y_test1)
    

1041    4.276666
862     3.610918
511     3.610918
327     1.791759
842     2.484907
          ...   
836     0.000000
451     4.672829
318     3.091042
884     3.433987
641     3.663562
Name: N, Length: 237, dtype: float64


In [42]:
print(predicted_valuesn)

[4.26813911 2.33492124 2.67410408 3.24290711 3.06222954 4.35691249
 4.66281596 4.36744319 2.92489698 2.8069665  3.26301994 2.74187717
 4.50811925 3.69131603 4.63186061 3.53647399 4.26147761 4.35691249
 3.0530411  3.93389247 2.74696052 4.21550371 3.20141227 3.45764239
 4.20927401 3.52201087 3.00503867 2.78943799 4.45884315 4.28497958
 3.62095457 3.71949164 2.71812767 3.27527336 4.66281596 3.12199185
 3.41497946 4.34802366 2.60196544 4.41355024 2.70805445 4.36485103
 3.69860576 4.50823709 2.84760906 2.23300541 2.7571532  3.24724176
 3.96567094 3.19207389 3.12199185 4.41902707 3.82461292 2.99773191
 4.34802366 3.08435073 2.96307145 4.36485103 3.02183756 4.36485103
 4.50312954 3.63832512 2.79441836 4.42306669 4.51808702 2.73288503
 2.80932887 4.3518093  4.53779    4.41355024 4.35691249 4.45413847
 3.28850474 3.09400822 1.80672842 4.43295608 2.68378628 4.3807964
 2.58109452 3.970707   3.04938376 3.78762561 3.73633558 3.58755
 2.78464779 3.78092225 3.18160409 4.37373986 4.3807964  3.35676362

In [44]:
pickle.dump(regressor1, open("modelngb.pkl", "wb"))
pickle.dump(regressor2, open("modelpgb.pkl", "wb"))
pickle.dump(regressor3, open("modelkgb.pkl", "wb"))

In [48]:
!python --version

Python 3.8.5
